<a href="https://colab.research.google.com/github/shabnamarasteh/AI_faceDetection/blob/arasteh/perisan_nlu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install hazm

In [ ]:
#  !pip install accelerate -U

In [ ]:
# %pip install nltk

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/test.csv',encoding = 'utf8')
data.drop_duplicates(keep='first')
df = data[['id','body','abstract']]

In [ ]:
df = df.rename(columns={'id':'id', 'text':'body', 'abstract':'abstract'})
df = df.dropna()

In [ ]:
print(df)

      id                                               body  \
0      1  به گزارش خبرگزاری فارس از تبریز، در هفته چهارم...   
1      2  خبرگزاری فارس مازندران ـ  نذری پزون یکی از سنت...   
2      3  به گزارش گروه بین‌الملل خبرگزاری فارس، «دونالد...   
3      4  به گزارش خبرگزاری فارس، آیت الله محمدتقی مصباح...   
4      5  به گزارش خبرگزاری فارس، «جمال موسیالا» در بازی...   
..   ...                                                ...   
493  495  پولاد عظیمی عضو هیأت اجرایی خانه کارگر دشت ورا...   
494  496  به گزارش خبرگزاری فارس، از هفته هفدهم رقابت‌ها...   
495  497  به گزارش خبرگزاری فارس از شهرکرد، فرهاد رئیسی ...   
496  498  به گزارش خبرگزاری فارس، از هفته هفدهم رقابت‌ها...   
497  499  به گزارش خبرگزاری فارس، از هفته چهاردهم رقابت‌...   

                                              abstract  
0    تیم مس سونگون برابر مهمان خود به برتری پرگل دس...  
1    پخت آش نذری در مرکز مازندران توسط یکی از عشاق ...  
2    رئیس جمهور آمریکا در پیامی در توییتر از تجمع ب...  
3    نماینده ول

In [ ]:
import hazm
normalizer = hazm.Normalizer()
df['abstract']= df['abstract'].apply(lambda x: normalizer.normalize(x))


In [ ]:
pd.set_option('display.max_colwidth',None)
df.head(1)
print(df['body'])

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
print(len(max(df['body'], key=len)))

30228


In [ ]:
%pip install datasets transformers rouge-score nltk -q

In [ ]:
%pip install torch

In [ ]:
import torch
import datasets
from datasets import Dataset
from datasets import load_metric

In [ ]:
import transformers
print(transformers.__version__)

4.35.2


In [ ]:
from transformers import AutoTokenizer

model_checkpoint ='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

pad_on_right = tokenizer.padding_side == "left"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
max_input_length = 1024
max_target_length = 128

In [ ]:
# df = df.sample(frac=1).reset_index(drop=True)
# train = df[:200]
# valid = df[200:450]
# test = df[450:]
# train.shape, valid.shape,test.shape

In [ ]:
train_size = int(0.6 * len(data))
test_size = int(0.2 * len(data))
val_size = len(data) - train_size - test_size

In [ ]:
train = data[:train_size]
test = data[train_size:train_size + test_size]
valid = data[train_size + test_size:]

In [ ]:
def preprocess_function(examples):
    inputs = ['summarize:' + doc for doc in examples["body"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["abstract"], max_length=max_target_length, truncation=True)
    print("-----------")
    print(labels["input_ids"])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
import torch
import datasets
from datasets import Dataset

train = Dataset.from_pandas(train)
valid = Dataset.from_pandas(valid)

In [ ]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_valid = valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/298 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


-----------
[[3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 673, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

-----------
[[3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 673, 2, 3704, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 586, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 10, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 10, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 10, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 10, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2,

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
batch_size = 5
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from nltk.collocations import *
from nltk.decorators import decorator, memoize
from nltk.featstruct import *
from nltk.grammar import *
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=False)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=False)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
  import gc
  gc.collect()

0

In [ ]:
# # determine the device we will be using for training
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# print("[INFO] training using {}".format(torch.cuda.get_device_name(0)))
# print('There are %d GPU(s) available.' % torch.cuda.device_count())

In [ ]:
torch.cuda.empty_cache()

In [ ]:
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-newsarticles",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=15,
    predict_with_generate=True,
    fp16=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# !pip install accelerate -U

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
metric = load_metric("rouge")

<ipython-input-34-7154407f1f07>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      | 236354 KiB | 236354 KiB | 236354 KiB |      0 B   |\n|       from large pool | 162560 KiB | 162560 KiB | 162560 KiB |      0 B   |\n|       from small pool |  73794 KiB |  73794 KiB |  73794 KiB |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         | 236354 KiB | 236354 KiB | 236354 KiB |      0 B   |\n|       from large pool | 162560 KiB | 162560 KiB |

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.170125,19.053700,15.588900,17.175100,17.187100,19.000000
2,No log,0.166264,19.053700,15.588900,17.175100,17.187100,19.000000
3,No log,0.166612,19.053700,15.588900,17.175100,17.187100,19.000000
4,No log,0.166038,19.053700,15.588900,17.175100,17.187100,19.000000
5,No log,0.162842,19.053700,15.588900,17.175100,17.187100,19.000000
6,No log,0.166639,19.053700,15.588900,17.175100,17.187100,19.000000
7,No log,0.163507,19.053700,15.588900,17.175100,17.187100,19.000000
8,No log,0.162718,19.053700,15.588900,17.175100,17.187100,19.000000
9,0.151800,0.163287,19.053700,15.588900,17.175100,17.187100,19.000000
10,0.151800,0.163491,19.053700,15.588900,17.175100,17.187100,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=900, training_loss=0.14608966403537327, metrics={'train_runtime': 429.5853, 'train_samples_per_second': 10.405, 'train_steps_per_second': 2.095, 'total_flos': 1209955705159680.0, 'train_loss': 0.14608966403537327, 'epoch': 15.0})

In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
eval_dataset = Dataset.from_pandas(test)

In [ ]:
eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

-----------
[[3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 1640, 3, 2, 3, 2, 431, 4433, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 10, 3, 2, 3, 2, 3, 2, 204, 3914, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 957, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 6503, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 10, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
predict_results = trainer.predict(
            eval_dataset,max_length=128, num_beams=3)

In [ ]:
metrics = predict_results.metrics

In [ ]:
metrics

{'test_loss': 0.1616283357143402,
 'test_rouge1': 56.4658,
 'test_rouge2': 54.4748,
 'test_rougeL': 55.1429,
 'test_rougeLsum': 55.2818,
 'test_gen_len': 106.3434,
 'test_runtime': 54.2878,
 'test_samples_per_second': 1.824,
 'test_steps_per_second': 0.368}

In [ ]:
predict_results.predictions

array([[0, 3, 2, ..., 3, 2, 3],
       [0, 3, 2, ..., 0, 0, 0],
       [0, 3, 2, ..., 0, 0, 0],
       ...,
       [0, 3, 2, ..., 3, 2, 3],
       [0, 3, 2, ..., 2, 3, 2],
       [0, 3, 2, ..., 3, 2, 3]])

In [ ]:
if args.predict_with_generate:
    predictions = tokenizer.batch_decode(predict_results.predictions, skip_special_tokens=False, clean_up_tokenization_spaces=True)
    print(predictions)
    predictions = [pred.strip() for pred in predictions]


['<pad> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> ', '<pad> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>', '<pad> <unk> <unk> <unk> <unk> <unk> <un

In [ ]:
predictions

['<pad> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>',
 '<pad> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<pad> <unk> <unk> <unk> <unk> <unk> <u

In [ ]:
test['abstract'][:2]

298    دبیر ستاد مبارزه با کرونا دانشگاه علوم پزشکی لرستان از ابتلای 60 هزار و 685 نفر از هم استانی‌ها به کرونا خبر داد و گفت: متاسفانه هزار و 289 نفر را به دلیل ابتلا به کووید 19 در استان از دست داده‌ایم.
299                                                                                                                                دستور جلسات و برنامه هیات اجرایی کمیته ملی المپیک برای روز یکشنبه مشخص شد.
Name: abstract, dtype: object

In [ ]:

import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install transformers
!pip install sentence-transformers

In [ ]:


from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-roberta-large')
sentence1 = tokenized_train[1].get("body")
sentence2 = tokenized_train[1].get("abstract")# encode sentences to get their embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print("Sentence 1:", sentence1)
print("Sentence 2:", sentence2)
print("Similarity score:", cosine_scores.item())

Sentence 1: خبرگزاری فارس مازندران ـ  نذری پزون یکی از سنت‌های دیرینه رایج در بین مردم کشور است که از گذشته تاکنون با نیت‌های مختلف و در شکل‌های متفاوت انجام می‌شود، این کار عموما در مناسبت‌های مختلف مذهبی صورت می‌گیرد.  پخت آش رشته نذری یکی از سنت‌هایی است که به وفور در استان مازندران به چشم می‌خورد، به این صورت که فرد بنا بر نیت خاصی در صورت حاجت روا شدن و یا بدون داشتن حاجتی به نیت یک شهید یا فرد خاصی ملزومات پخت آش را تهیه و اقدام به پخت و توزیع آن می‌کند.  همانطور که می‌دانید ۱۳ دی ماه که تا سال گذشته یک روز معمولی در تقویم کشورمان بوده است امسال تبدیل به یکی از مناسبت‌های جهانی شده، در این روز، سردار سپهبد حاج قاسم سلیمانی و همرزمانش در حمله هلی‌کوپتری آمریکایی‌ها در فرودگاه بغداد به شهادت رسید و ملتی را سیاه‌پوش کرد.  در اولین سالگرد شهادت شهید حاج قاسم این چهره بین‌المللی و ملقب به سردار دل‌ها مردم زیادی در سطح جهان و کشور اقدام به توزیع نذری کردند، پخت آش نذری در مرکز مازندران توسط یکی از عشاق سردار آن هم تنها به نیت گرامیداشت سالگرد شهادت سردار سلیمانی یکی از اقدامات در این م